In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import json

In [10]:
%run GetLossObj.ipynb

In [22]:
baseDir = "../save/server1/rnn/model45/"
# expData = torch.load(baseDir+"experimentData.pth")
plotTrainReconLosses = torch.load(baseDir + "plot_train_recon_losses", map_location="cpu")
plotValReconLosses = torch.load(baseDir + "plot_val_recon_losses", map_location="cpu")
with open(baseDir + "args.txt") as f:
    args = f.read()

In [23]:
args

'{"h_dim": 256, "z_dim": 0, "no_cuda": false, "no_attn": true, "n_epochs": 20, "batch_size": 64, "n_layers": 2, "initial_lr": 1e-05, "lr_decay_every": 10, "lr_decay_factor": 0.1, "lr_decay_beginning": 20, "print_every": 1, "criterion": "L1Loss", "save_freq": 10, "down_sample": 0.0, "model": "rnn", "lambda_l1": 0, "lambda_l2": 0.0005, "no_schedule_sampling": false, "scheduling_start": 1.0, "scheduling_end": 0.0, "tries": 12, "kld_warmup_until": 5, "kld_weight_max": 0.1, "no_shuffle_after_epoch": false, "clip": 10, "dataset": "human", "predictOnTest": true, "encoder_input_dropout": 0.5, "encoder_layer_dropout": 0.5, "decoder_input_dropout": 0.5, "decoder_layer_dropout": 0.5, "noEarlyStopping": false, "earlyStoppingPatients": 3, "earlyStoppingMinDelta": 0.0001, "bidirectionalEncoder": true, "local": false, "cuda": true, "_device": "cuda", "use_attn": false, "use_schedule_sampling": true, "x_dim": 16, "input_sequence_len": 25, "target_sequence_len": 50, "channels": 2, "output_dim": 32, "sa

In [24]:
args = Bunch(json.loads(args))
args.cuda = False

In [25]:
print(args.initial_lr, args.dataset)

1e-05 human


In [6]:
dataDict = getDataLoaders("/Users/danielzeiberg/Documents/Data/Human/Processed/INPUT_HORIZON_25_PREDICTION_HORIZON_50/")

INFO:root:Getting data from /Users/danielzeiberg/Documents/Data/Human/Processed/INPUT_HORIZON_25_PREDICTION_HORIZON_50/
INFO:root:Getting train loader
INFO:root:Getting val loader
INFO:root:Getting test loader


In [11]:
getlossobj = PostProcess(baseDir, dataDict=dataDict, chooseModel="rnn")

In [12]:
print("min val loss: {:.4f} at idx {}".format(np.min(plotValReconLosses), np.argmin(plotValReconLosses)))

min val loss: 10.2568 at idx 19


In [13]:
valRetVals = getlossobj.getLossAtEpoch("rnn_full_model.pth", "val")

In [14]:
trainRetVals = getlossobj.getLossAtEpoch("rnn_full_model.pth", "train")

In [15]:
testRetVals = getlossobj.getLossAtEpoch("rnn_full_model.pth", "test")

In [16]:
print("train: {:.4f}, val: {:.4f}, test: {:.4f}".format(trainRetVals["L1Loss"],valRetVals["L1Loss"], testRetVals["L1Loss"]))

train: 10.1529, val: 10.2568, test: 10.0944


In [ ]:
def plotRandomInstance(preds,targets, datas, dataset):
    instance = np.random.randint(datas.shape[0])
    sensor = np.random.randint(datas.shape[2])
    p = preds[instance, :, sensor]
    t = targets[instance, :, sensor]
    d = datas[instance, :, sensor]
    plt.plot(range(5,65, 5),p, label="Pred")
    #plt.fill_between(range(0,60, 5),m-1.96*std, m+1.96*std)
    plt.plot(range(5,65, 5),t, label="target")
    plt.plot(range(-55,5, 5), d, label="input")
    plt.legend()
    plt.xlabel("Minutes relative to prediction time")
    plt.ylabel("Speed (mph)")
    plt.title("random sensor sample {}".format(dataset))
    plt.show()

In [ ]:
plotRandomInstance(trainRetVals["preds"], trainRetVals["targets"], trainRetVals["datas"], "Train")

In [ ]:
plotRandomInstance(valRetVals["preds"], valRetVals["targets"], valRetVals["datas"], "Validation")

In [ ]:
plotRandomInstance(testRetVals["preds"], testRetVals["targets"], testRetVals["datas"], "Test")

In [ ]:
total = 0
for p in getlossObj.model.parameters():
    total += np.prod(p.shape)
print(total)

In [ ]:
print(getlossObj.dataDict["train_mean"], getlossObj.dataDict["val_mean"], getlossObj.dataDict["train_std"], getlossObj.dataDict["val_std"])

In [ ]:
plt.plot(np.sqrt(np.mean((predsVal - targetsVal)**2, axis=(1,2))))
ticks = plt.xticks(np.arange(0,12), np.arange(5,13*5, 5))
plt.xlabel("Minutes after prediction time")
plt.ylabel("RMSE")


In [ ]:
plotTrainValCurve(plotTrainReconLosses, plotValReconLosses)

In [ ]:
next(iter(getlossobj.model.parameters())).is_cuda

In [ ]:
for name, p in getlossobj.model.named_parameters():
    if bias in 
    print(name)